In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
styles = ['seaborn-whitegrid', 'seaborn-poster']

from stimator import read_model, Solutions

# Sistema dos glioxalases em *L.infantum*

## Exemplo de análise de sensibilidade

Parte da publicação:

Sousa Silva, M. , Ferreira, A.E.N., Tomás, A.M., Cordeiro, C., Ponces Freire, A. (2005)  Quantitative assessment of the glyoxalase pathway in Leishmania infantum as a therapeutic target by modelling and computer simulation. *FEBS Journal* **272(10)**: 2388-2398.

![Glyoxalases](images/glos_small.png)

### Componentes:

- Formação não enzimática de metilglioxal (MG) a partir dos fosfatos de triose da glicólise
- Conjugação não enzimática (e reversível) com tripanotiono (T(SH)2)
- Ação enzimática de Glioxalase I e de Glioxalase II, regenerando o tripanotiono e formando D-lactato.


### Objetivo:

Prever se algumas modificações experimentais da via

- inibições de enzimas
- aumento da formação de MG
- interferência com a biosíntese de tiois

poderá **aumentar** o estado estacionário de metilglioxal em células de *L. infantum*.


In [ ]:
m = read_model("""
title Glyoxalase system

vgen       : -> MG,            k1 * GAP + k2 * DHAP
vconjf     : MG + TSH2 -> HTA, k3 * MG * TSH2
vconjr     : HTA -> MG + TSH2, k4 * HTA
vglo1      : HTA -> SDLTSH,    Vglo1 * HTA / (Kmglo1 + HTA)
vglo2      : SDLTSH -> TSH2,   Vglo2 * SDLTSH  / (Kmglo2 + SDLTSH)

k1          = 6.4e-3          # min-1
k2          = 6.6e-4          # min-1
GAP         = 0.0072          # mM
DHAP        = 0.16            # mM

k3          = 0.34            # mM-1 min-1
k4          = 1.01            # min-1

Vglo1       = 2 * 3.042       # mM min-1
Vglo2       = 2 * 2.653       # mM min-1
Kmglo1      = 2 * 0.253       # mM
Kmglo2      = 2 * 0.0980      # mM

init: (TSH2 = 2 * 0.45)
!! HTA SDLTSH MG
""")

In [ ]:
m.solve(tf=40.0).plot(style=styles)

Para avaliar o efeito do aumento do geração não enzimática de MG, introduzimos no modelo um factor, `fgen`, inicialmente igual a 1, que expressa variações relativas na velocidade `vgen` (considerada constante durante uma simulação).

Desta forma podemos executar uma série de simulações para diferentes valores de `fgen`:

In [ ]:
m_fgen = read_model("""
title Glyoxalase system

vgen       : -> MG,            fgen * k1 * GAP + fgen * k2 * DHAP
vconjf     : MG + TSH2 -> HTA, k3 * MG * TSH2
vconjr     : HTA -> MG + TSH2, k4 * HTA
vglo1      : HTA -> SDLTSH,    Vglo1 * HTA / (Kmglo1 + HTA)
vglo2      : SDLTSH -> TSH2,   Vglo2 * SDLTSH  / (Kmglo2 + SDLTSH)

fgen        = 1
k1          = 6.4e-3          # min-1
k2          = 6.6e-4          # min-1
GAP         = 0.0072          # mM
DHAP        = 0.16            # mM

k3          = 0.34            # mM-1 min-1
k4          = 1.01            # min-1

Vglo1       = 2 * 3.042       # mM min-1
Vglo2       = 2 * 2.653       # mM min-1
Kmglo1      = 2 * 0.253       # mM
Kmglo2      = 2 * 0.0980      # mM
totalSH     = 2 * 0.45
init: (TSH2 = 2 * 0.45)
!! HTA SDLTSH MG
""")

sols = Solutions()

In [ ]:
print('fgen MG')

for fgen in 1, 2, 3, 4:
    m_fgen.parameters.fgen = fgen
    
    s = m_fgen.solve(tf=40.0)
    print (fgen, s.last['MG'])
    sols += s

In [ ]:
sols.plot(fig_size=(16,12), yrange=(0,0.003), titles=[f'fgen = {fgen}' for fgen in (1, 2, 3, 4)], style=styles)
m_fgen.parameters.fgen=1

A concentração "final" (estado estacionário) de MG parece ser proporcional a `fgen`.

Podemos aplicar a mesma técnica e avaliar o efeito de se variar:

- a formação não enzimática de MG (`fgen`)
- a atividade de GLO I (`fglo1`)
- a atividade de GLO II (`fglo2`)
- a concentração **total** de tiois, HTA + TSH2 + SDLTSH (`fSH`)

In [ ]:
m_fs = read_model("""
title Glyoxalase system

vgen       : -> MG,            fgen * k1 * GAP + fgen * k2 * DHAP
vconjf     : MG + TSH2 -> HTA, k3 * MG * TSH2
vconjr     : HTA -> MG + TSH2, k4 * HTA
vglo1      : HTA -> SDLTSH,    fglo1 * Vglo1 * HTA / (Kmglo1 + HTA)
vglo2      : SDLTSH -> TSH2,   fglo2 * Vglo2 * SDLTSH  / (Kmglo2 + SDLTSH)

fgen        = 1
fglo1       = 1
fglo2       = 1
fSH         = 1

k1          = 6.4e-3          # min-1
k2          = 6.6e-4          # min-1
GAP         = 0.0072          # mM
DHAP        = 0.16            # mM

k3          = 0.34            # mM-1 min-1
k4          = 1.01            # min-1

Vglo1       = 2 * 3.042       # mM min-1
Vglo2       = 2 * 2.653       # mM min-1
Kmglo1      = 2 * 0.253       # mM
Kmglo2      = 2 * 0.0980      # mM

init: (TSH2 = 2 * 0.45)
!! HTA SDLTSH MG
""")

In [ ]:
MG_ref = m_fs.solve(tf=40.0).last['MG']

ratios = np.concatenate((np.linspace(0.05, 1, 20), np.linspace(1, 2, 20)))

s = m_fs.scan({'fgen':ratios}, tf=40.0)
MGs_gen = [solution.last['MG'] / MG_ref for solution in s]

s = m_fs.scan({'fglo1':ratios}, tf=40.0)
MGs_glo1 = [solution.last['MG'] / MG_ref for solution in s]
    
s = m_fs.scan({'fglo2':ratios}, tf=40.0)
MGs_glo2 = [solution.last['MG'] / MG_ref for solution in s]

ratiosSH = m_fs.init.TSH2 * ratios
s = m_fs.scan({'init.TSH2':ratiosSH}, tf=40.0)
MGs_SH = [solution.last['MG'] / MG_ref for solution in s]

In [ ]:
colors = sns.color_palette('tab10', 4)
with sns.axes_style("whitegrid"):
    with sns.plotting_context("notebook", font_scale=1.5):
        f, ax = plt.subplots(figsize=(8,8))
        ax.set_aspect(1)
        ax.plot(ratios, MGs_gen,  color=colors[0], label="gen  MG", marker='o')
        ax.plot(ratios, MGs_glo1, color=colors[1], label="GLO I", marker='o')
        ax.plot(ratios, MGs_glo2, color=colors[2], label="GLO II", marker='o')
        ax.plot(ratios, MGs_SH,   color=colors[3], label="-SH total", marker='o')
        ax.set(xlabel="$p/p_{ref}$", ylabel="MG / MGref", ylim=(0,2))
        ax.plot([1],[1], 'ko', ms=10)
        ax.legend(loc='upper center')

In [ ]:
f